In [4]:
### Illustration du biais de regularisation
### 26 septembre 2019
### Jeremy LHour

library("MASS")
library("glmnet")

# fonctions utilisateur
source("functions/DataSim.R") 

set.seed(12071990)
n = 2000
data = DataSim(n, p=20, Ry= .2, Intercept=F)
y = data$y; X = data$X

Warning message in c * gamma:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in c * b:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”

In [8]:
nopen = c(1,3)
phi = rep(1,ncol(X))
phi[nopen] = 0
cv.lasso = cv.glmnet(X,y, family="gaussian",alpha=1,penalty.factor=phi,nfolds=5)
b.lasso = coef(cv.lasso)
b.lasso = b.lasso[-1] # on enleve la constante
s.hat = (b.lasso != 0) # indices des variables actives
print(s.hat)

21 x 1 sparse Matrix of class "dgCMatrix"
                      1
(Intercept)  0.31123998
V1          -0.57141236
V2           .         
V3          -0.02395663
V4           .         
V5           .         
V6           .         
V7           .         
V8           .         
V9           .         
V10          .         
V11          .         
V12          .         
V13          .         
V14          .         
V15          .         
V16          .         
V17          .         
V18          .         
V19          .         
V20          .         
21 x 1 sparse Matrix of class "lgCMatrix"
            1
(Intercept) |
V1          |
V2          .
V3          |
V4          .
V5          .
V6          .
V7          .
V8          .
V9          .
V10         .
V11         .
V12         .
V13         .
V14         .
V15         .
V16         .
V17         .
V18         .
V19         .
V20         .


In [ ]:
PostLasso <- function(X,y,nopen){
    #'@param nopen indices des variables a ne pas penaliser
    phi = rep(1,ncol(X))
    phi[nopen] = 0
    cv.lasso = cv.glmnet(X,y, family="gaussian",alpha=1,penalty.factor=phi,nfolds=5)
    s.hat = (coef(cv.lasso) != 0)
    postlasso = lm(y ~ )
}